In [ ]:
!pip install tensorflow
!pip install tqdm
!pip install gradio

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings('ignore')
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
# Importing ResNet50,preprocess_input and Model from the necessary modules:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers import TextVectorization
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
#Importing tqdm from the necessary modules:
from tqdm import tqdm
from collections import Counter
import pickle

In [ ]:
IMAGES_PATH = "/content/drive/MyDrive/data/Images"
CAPTIONS_PATH = "/content/drive/MyDrive/data/captions.txt"

In [ ]:
IMAGE_SIZE = (224, 224)
SEQ_LENGTH = 25
VOCAB_SIZE = 10000
EMBED_DIM = 512
FF_DIM = 512
BATCH_SIZE = 128
EPOCHS = 20

In [ ]:
#features = {}
#BASE_DIR = os.getcwd()
#directory = os.path.join(BASE_DIR, '/content/drive/MyDrive/data/Images')

#for img_name in tqdm(os.listdir(directory)):
 #   img_path = os.path.join(directory, img_name)
  #  image = load_img(img_path, target_size=(224, 224))
#
 #   image = img_to_array(image)
  #  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#
 #   image = preprocess_input(image)
#
 #   feature = model.predict(image, verbose=0)
#
 #   image_id = img_name.split('.')[0]
  #  features[image_id] = feature
#WORKING_DIR = BASE_DIR
#pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb'))

In [ ]:
with open(os.path.join('/content/drive/MyDrive/working/features.pkl'), 'rb') as f:
    features = pickle.load(f)
    print(f"Number of extracted features: {len(features)}")


Number of extracted features: 8091


In [ ]:
# Loads captions (text) data and maps them to corresponding images.
def load_captions_data(filename):
    with open(filename) as caption_file:
        caption_data = caption_file.readlines()[1:]
        caption_mapping = {}
        text_data = []
        images_to_skip = set()

        for line in caption_data:
            line = line.rstrip("\n")
            # Each image is repeated five times for the five different captions.# Image name and captions are separated using a comma
            img_name, caption = line.split(",", 1)
            img_name = os.path.join(IMAGES_PATH, img_name.strip())
            # Removing caption that are either too short to too long
            tokens = caption.strip().split()
            if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                images_to_skip.add(img_name)
                continue

            if img_name.endswith("jpg") and img_name not in images_to_skip:# A start and an end token must be added to each caption
                caption = "<start> " + caption.strip() + " <end>"
                text_data.append(caption)

                if img_name in caption_mapping:
                    caption_mapping[img_name].append(caption)
                else:
                    caption_mapping[img_name] = [caption]

        for img_name in images_to_skip:
            if img_name in caption_mapping:
                del caption_mapping[img_name]

        return caption_mapping, text_data# Splits the dataset into training, validation, and test sets
def train_val_split(caption_data, validation_size=0.2, test_size=0.05, shuffle=True):
    all_images = list(caption_data.keys())
    if shuffle:
        np.random.shuffle(all_images)

    # First split to get test set
    train_val_keys, test_keys = train_test_split(all_images, test_size=test_size, random_state=42)
    # Then split remaining into train and validation
    train_keys, validation_keys = train_test_split(train_val_keys, test_size=validation_size, random_state=42)

    training_data = {img_name: caption_data[img_name] for img_name in train_keys}
    validation_data = {img_name: caption_data[img_name] for img_name in validation_keys}
    test_data = {img_name: caption_data[img_name] for img_name in test_keys}

    return training_data, validation_data, test_data

# Loading the dataset
captions_mapping, text_data = load_captions_data(CAPTIONS_PATH)
captions_mapping, text_data = load_captions_data(CAPTIONS_PATH)

# Spliting the dataset
train_data, validation_data, test_data = train_val_split(captions_mapping)
print(f"Total number of samples: {len(captions_mapping)}")
print(f"----> Number of training samples: {len(train_data)}")
print(f"----> Number of validation samples: {len(validation_data)}")
print(f"----> Number of test samples: {len(test_data)}")

Total number of samples: 7643
----> Number of training samples: 5808
----> Number of validation samples: 1452
----> Number of test samples: 383


In [ ]:
def custom_standardization(input_string):
    # Lowercasing all of the captions
    lowercase = tf.strings.lower(input_string)
    # Charecters to remove
    strip_chars = "!\"#$%&'()*+,-./:;=?@[\]^_`{|}~1234567890"
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# Defining the vectorizer
vectorization = TextVectorization(
    # Number of unique tokens
    max_tokens=VOCAB_SIZE,
    output_mode="int",# Maximum length of captions. Padding tokens(zeros) will be added to shorter captions.
    output_sequence_length=SEQ_LENGTH,
    # Standardizing the captions
    standardize=custom_standardization)

# Adapting the vectorizer with the captions
vectorization.adapt(text_data)

# Data augmentation for image data
image_augmentation = keras.Sequential([layers.RandomFlip("horizontal"),
                                       layers.RandomRotation(0.2),
                                       layers.RandomContrast(0.3)])

# Standardizing the text data
text_data = list(map(lambda x: str(custom_standardization(x).numpy())[2:-1], text_data))

In [ ]:
def visualaization(data, num_of_images):
    count = 1
    fig = plt.figure(figsize=(10,20))
    for filename in list(data.keys())[100:100+num_of_images]:
        captions = list(map(lambda x: str(custom_standardization(x).numpy())[2:-1], data[filename]))
        image_load = load_img(filename, target_size=(199,199,3))

        ax = fig.add_subplot(num_of_images,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1

        ax = fig.add_subplot(num_of_images,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0, 1)  # Changed y-limit for single caption
        ax.text(0, 0, captions[0], fontsize=20)
        count += 1
    plt.show()

visualaization(train_data, 7) # Make sure to call the new function name

In [ ]:
def captions_length(data):
    plt.figure(figsize=(15, 7), dpi=300)
    sns.set_style('darkgrid')
    sns.histplot(x=[len(x.split(' ')) for x in data], kde=True, binwidth=1)
    plt.title('Captions length histogram', fontsize=15, fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.xlabel('Length', fontweight='bold')
    plt.ylabel('Freaquency', fontweight='bold')
    plt.show()

captions_length(text_data)

In [ ]:
def word_occurrences(data):
    # Combining all sentences into a single string
    all_text = ' '.join(data)
    all_text = all_text.replace('a ', '')
    all_text = all_text.replace('<start> ', '')
    all_text = all_text.replace('<end> ', '')
    # Splitting the text into words and count occurrences
    word_counts = Counter(all_text.split())

    words = list(word_counts.keys())[:30]
    values = list(word_counts.values())[:30]
   # Normalize values to be between 0 and 1
    normalized_values = np.array(values) / np.max(values)
    colors = np.array(['rgba(30, 58, 138, {})'.format(0.4 + 0.5 * (value)) for value in normalized_values])

    fig = go.Figure(data=[go.Pie(labels=words, values=values, hole=.6, marker=dict(colors=colors), textinfo='label')])

    fig.update_layout(title_text='Word occurrences in captions (except for letter \'a\')', title_font=dict(size=23, family='Balto'))

    fig.show()

word_occurrences(text_data)

In [ ]:
def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, captions):
    # Processed images: (None, 299, 299, 3), Vectorized captions: (None, None, 25)
    return decode_and_resize(img_path), vectorization(captions)

# Prepares the dataset
def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    dataset = dataset.map(process_input, num_parallel_calls=tf.data.AUTOTUNE)
    # Prefetching the next batch of data based on available resources while the current batch is being processed.
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return dataset

# Making the datasets by passing the list of images and the list of corresponding captions
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))
validation_dataset = make_dataset(list(validation_data.keys()), list(validation_data.values()))

In [ ]:
def get_cnn_model():
    base_model = efficientnet.EfficientNetB7(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights="imagenet")
    # فتح آخر 20 طبقة للتدريب لتحسين الأداء
    for layer in base_model.layers[-20:]:
        layer.trainable = True
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model

class MLPMixerEncoder(layers.Layer):
    def __init__(self, embed_dim, tokens_mlp_dim, channels_mlp_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_proj = layers.Dense(embed_dim)

        self.mixer_blocks = keras.Sequential([
            keras.Sequential([
                layers.LayerNormalization(),
                layers.Permute((2, 1)),
                layers.Dense(tokens_mlp_dim, activation='gelu'),
                layers.Dense(49),
                layers.Permute((2, 1)),
                layers.LayerNormalization(),
                layers.Dense(channels_mlp_dim, activation='gelu'),
                layers.Dense(embed_dim),
            ]) for _ in range(2)  # عدد أكبر من MLP Mixer blocks
        ])

    def call(self, inputs):
        if inputs.shape[-1] != EMBED_DIM:
            inputs = self.embed_proj(inputs)

        x = inputs
        for block in self.mixer_blocks.layers:
            residual = x
            x = block(x)
            x = x + residual

        return x

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens *= self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

class MLPMixerDecoder(layers.Layer):
    def __init__(self, embed_dim, seq_len, vocab_size, token_mlp_dim, channel_mlp_dim):
        super().__init__()
        self.embedding = PositionalEmbedding(embed_dim=embed_dim, sequence_length=seq_len, vocab_size=vocab_size)
        self.mlp1 = keras.Sequential([
            layers.LayerNormalization(),
            layers.Dense(token_mlp_dim, activation='gelu'),
            layers.Dense(embed_dim),
            layers.Dropout(0.2)
        ])
        self.mlp2 = keras.Sequential([
            layers.LayerNormalization(),
            layers.Dense(channel_mlp_dim, activation='gelu'),
            layers.Dense(embed_dim),
            layers.Dropout(0.2)
        ])

        self.cross_proj = layers.Dense(embed_dim)
        self.out = layers.Dense(vocab_size, activation="softmax")

    def call(self, inputs, encoder_outputs, training=False, mask=None):
        x = self.embedding(inputs)
        residual = x

        x = self.mlp1(x) + residual  # Token mixing
        x = self.mlp2(x) + x         # Channel mixing

        encoder_proj = self.cross_proj(encoder_outputs)
        encoder_proj = tf.reduce_mean(encoder_proj, axis=1, keepdims=True)

        x = x + encoder_proj
        preds = self.out(x)
        return preds

cnn_model = get_cnn_model()
encoder = MLPMixerEncoder(
    embed_dim=EMBED_DIM,
    tokens_mlp_dim=256,
    channels_mlp_dim=FF_DIM
)

# باقي كلاس ImageCaptioningModel لم يتغير، نحتفظ به كما هو
class ImageCaptioningModel(keras.Model):
    def __init__(self, cnn_model, encoder, decoder, num_captions_per_image=5, image_aug=None):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

        print()
        print(f'CNN input shape: {cnn_model.input_shape}')
        print(f'CNN output shape: {cnn_model.output_shape}', end='\n'*2)
        print(f'Encoder input ---> Dense layer shape: {cnn_model.output_shape} ---> (None, {cnn_model.output_shape[1]}, {EMBED_DIM})')
        print(f'Encoder output shape: (None, {cnn_model.output_shape[1]}, {EMBED_DIM})', end='\n'*2)
        print(f'Decoder input 1 (Caption) ---> Positional Embedding shape: (None, {SEQ_LENGTH-1}) ---> (None, {SEQ_LENGTH-1}, {EMBED_DIM})')
        print(f'Decoder input 2 (Embedded image features) shape: (None, {cnn_model.output_shape[1]}, {EMBED_DIM})')
        print(f'Decoder output (MH Cross-Attention) shape: (None, {SEQ_LENGTH-1}, {EMBED_DIM})')
        print(f'Decoder prediction (Dense layer) shape: (None, {SEQ_LENGTH-1}, {VOCAB_SIZE})')

         # Calculates the loss, taking into account a mask to handle padding.
    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    # Calculates the accuracy, taking into account a mask to handle padding.
    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        # Creating a binary mask where 1 indicates a valid token, and 0 indicates padding.
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(batch_seq_inp, encoder_out, training=training, mask=mask)
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)

        return loss, acc

    # Iterates through each caption for the given image, computes loss and accuracy, updates weights, and trackers.
    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0
         # Applies image augmentation if image_aug is provided.
        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(img_embed, batch_seq[:, i, :], training=True)
                 # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (self.encoder.trainable_variables + self.decoder.trainable_variables)

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(),
                "acc": self.acc_tracker.result()}

    # Similar to train_step but without updating weights.
    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(img_embed, batch_seq[:, i, :], training=False)

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(),
                "acc": self.acc_tracker.result()}
    @property
    def metrics(self):
        # We must list the metrics here so the `reset_states()` can be,
                                                  # called automatically.
        return [self.loss_tracker, self.acc_tracker]
decoder = MLPMixerDecoder(embed_dim=EMBED_DIM, seq_len=SEQ_LENGTH, vocab_size=VOCAB_SIZE,
                          token_mlp_dim=256, channel_mlp_dim=FF_DIM)

caption_model = ImageCaptioningModel(cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation)

cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)




In [ ]:
# Defining the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

# EarlyStopping criteria
# Training will stop if there is no improvement in the validation loss for 3 consecutive epochs.
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor="val_loss",
    mode="min"
)

# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate)

# Creating a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compiling the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

# Training the model
history = caption_model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, callbacks=[early_stopping])

In [ ]:
plt.figure(figsize=(15, 7), dpi=200)
sns.set_style('whitegrid')
plt.plot([x+1 for x in range(len(history.history['loss']))], history.history['loss'], color='#004EFF', marker='o')
plt.plot([x+1 for x in range(len(history.history['loss']))], history.history['val_loss'], color='#00008B', marker='h')
plt.title('Train VS Validation', fontsize=15, fontweight='bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel('Epoch', fontweight='bold')
plt.ylabel('Loss', fontweight='bold')
plt.legend(['Train Loss', 'Validation Loss'], loc='best')
plt.show()

In [ ]:
vocab = vectorization.get_vocabulary()
INDEX_TO_WORD = {idx: word for idx, word in enumerate(vocab)}
MAX_DECODED_SENTENCE_LENGTH = SEQ_LENGTH - 1
test_images = list(test_data.keys())

def beam_search_algorithm(image_path, beam_width=3):
    image = decode_and_resize(image_path)
    image = tf.expand_dims(image, 0)
    image_features = caption_model.cnn_model(image)
    encoder_output = caption_model.encoder(image_features, training=False)

    sequences = [[list(), 0.0]]  # [(sequence, score)]
    for _ in range(MAX_DECODED_SENTENCE_LENGTH):
        all_candidates = []
        for seq, score in sequences:
            caption_input = "<start> " + " ".join(seq)
            tokenized_input = vectorization([caption_input])[:, :-1]
            mask = tf.math.not_equal(tokenized_input, 0)
            preds = caption_model.decoder(tokenized_input, encoder_output, training=False, mask=mask)
            preds = preds[0, len(seq), :]
            top_k = np.argsort(preds)[-beam_width:]

            for idx in top_k:
                word = INDEX_TO_WORD[idx]
                if word == "<end>":
                    all_candidates.append((seq, score))  # Don't add new word, just keep the same
                    continue
                candidate = seq + [word]
                candidate_score = score - np.log(preds[idx] + 1e-10)
                all_candidates.append((candidate, candidate_score))

        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:beam_width]

    final_caption = " ".join(sequences[0][0])
    return final_caption


In [ ]:
# Generating captions
from tqdm.notebook import tqdm

generated_captions = {}

pbar = tqdm(total=len(test_data), position=0, leave=True, colour='green')
for image_id in test_data:
    cap = beam_search_algorithm(image_id)
    generated_captions[image_id] = cap
    pbar.update(1)

pbar.close()

  0%|          | 0/383 [00:00<?, ?it/s]

In [ ]:
import pickle
# حفظ القاموس في ملف pkl
with open('/content/drive/MyDrive/working/generated_captions.pkl', 'wb') as f:
    pickle.dump(generated_captions, f)


In [ ]:

with open('/content/drive/MyDrive/working/generated_captions.pkl', 'rb') as f:
    generated_captions = pickle.load(f)


In [ ]:
# Calculates BLEU score of predictions
def BLEU_score(actual, predicted):
    # Standardizing the actual captions
    processed_actual = []
    for i in actual:
        cap = [INDEX_TO_WORD[x] for x in vectorization(i).numpy() if INDEX_TO_WORD[x] != '']
        cap = ' '.join(cap)
        processed_actual.append(cap)

    # Calculating the BLEU score by comparing the predicted caption with five actual captions.
    b1=corpus_bleu(processed_actual, predicted, weights=(1.0, 0, 0, 0))
    b2=corpus_bleu(processed_actual, predicted, weights=(0.5, 0.5, 0, 0))
    b3=corpus_bleu(processed_actual, predicted, weights=(0.3, 0.3, 0.3, 0))
    b4=corpus_bleu(processed_actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

    return [
        (f'BLEU-4: {round(b4, 5)}'),
        (f'BLEU-3: {round(b3, 5)}'),
        (f'BLEU-2: {round(b2, 5)}'),
        (f'BLEU-1: {round(b1, 5)}'),
        (f'Generated Caption: {predicted[0]}'),
    ]


In [ ]:
def visualization(data, generated_captions, evaluator, num_of_images):
    keys = list(data.keys()) # List of all test images
    images = [np.random.choice(keys) for i in range(num_of_images)] # Randomly selected images

    count = 1
    fig = plt.figure(figsize=(6,20))
    for filename in images:
        actual_cap = data[filename]
        actual_cap = [x.replace("<start> ", "") for x in actual_cap] # Removing the start token
        actual_cap = [x.replace(" <end>", "") for x in actual_cap] # Removing the end token

        caption = generated_captions[filename]
        caps_with_score = evaluator(actual_cap, [caption]*(len(actual_cap)))

        image_load = load_img(filename, target_size=(199,199,3))
        ax = fig.add_subplot(num_of_images,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1

        ax = fig.add_subplot(num_of_images,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0,len(caps_with_score))
        for i, text in enumerate(caps_with_score):
            ax.text(0,i,text,fontsize=10)
        count += 1
    plt.show()

visualization(test_data, generated_captions, BLEU_score, 7)


In [ ]:
captions_length(list(generated_captions.values()))
word_occurrences(list(generated_captions.values()))


In [ ]:
from matplotlib import pyplot as plt
from tensorflow.keras.utils import load_img
import numpy as np

image_path = "/content/drive/MyDrive/data/Images/233270519_d60d4518fa.jpg"

beam_widths = [1]

plt.figure(figsize=(8, 8))
img = load_img(image_path)
plt.imshow(img)
plt.axis('off')

captions = []
for width in beam_widths:
    caption = beam_search_algorithm(image_path, beam_width=width)
    captions.append((width, caption))
    print(f"Beam width = {width}: {caption}")

title_text = "\n".join([f"Beam {w}: {c}" for w, c in captions])
plt.title(title_text, fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
import gradio as gr

def generate_captions_gradio(image):
    # حفظ الصورة المؤقتة
    temp_path = "/content/temp_input.jpg"
    image.save(temp_path)

    # توليد الكابشنات
    beam_widths = [1]
    captions = []
    for width in beam_widths:
        caption = beam_search_algorithm(temp_path, beam_width=width)
        captions.append(f"Beam {width}: {caption}")

    # تجميع الكابشنات
    final_caption = "\n".join(captions)
    return image, final_caption

# واجهة Gradio
interface = gr.Interface(
    fn=generate_captions_gradio,
    inputs=gr.Image(type="pil", label="ارفع صورة"),
    outputs=[
        gr.Textbox(label="Generated caption "),
        gr.Textbox(label="BLEU Score")
    ],
    title="Caption generator from images with MLPMixer algorithm",
    description="This image captioning model uses a CNN (e.g., EfficientNet) as an image encoder to extract visual features, followed by an MLP-Mixer-based decoder that generates natural language captions"
)

# تشغيل الواجهة
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e4018341c28e7076d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
